# Cognitive testing & LLM biases
This notebook shows how to use `edsl` to investigate whether LLMs demonstrate bias towards content that they generate or improve compared with content generated or improved by other LLMs. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/explore_llm_biases.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionFreeText, QuestionLinearScale
from edsl import Agent, Model

In [3]:
Model.available()

['claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'dbrx-instruct',
 'gemini_pro',
 'gpt-3.5-turbo',
 'gpt-4-1106-preview',
 'llama-2-13b-chat-hf',
 'llama-2-70b-chat-hf',
 'mixtral-8x7B-instruct-v0.1']

We select a set of models and instruct them to generate some content:

In [4]:
models = [
    Model('gpt-3.5-turbo', cache=False), 
    Model('gpt-4-1106-preview', cache=False), 
    Model('mixtral-8x7B-instruct-v0.1', cache=False)
]

In [5]:
q_example = QuestionFreeText(
    question_name = "example",
    question_text = "Draft a detailed executive summary for a resume for an average software engineer."
)

In [6]:
r_example = q_example.by(models).run()

In [7]:
r_example.select('model.model','answer.example').print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ model                      ┃ answer                                                                             ┃
┃ .model                     ┃ .example                                                                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt-3.5-turbo              │ As a seasoned software engineer with 5 years of experience, I have a proven track  │
│                            │ record of designing and implementing complex software solutions. Skilled in        │
│                            │ multiple programming languages such as Java, Python, and JavaScript, I have        │
│                            │ successfully delivered projects on time and within budget. With a strong           │
│                            │ background in software development methodologies and a passion for innovation, I   │
│                            │ am eager to contribute my expertise to a dynamic team.                             │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ gpt-4-1106-preview         │ Executive Summary: A dedicated and results-driven software engineer with a proven  │
│                            │ track record of designing, implementing, and maintaining complex software systems. │
│                            │ Possesses a strong foundation in computer science principles, coupled with         │
│                            │ hands-on experience in full-stack development, database management, and cloud      │
│                            │ technologies. Adept at collaborating with cross-functional teams and excelling in  │
│                            │ agile environments. Brings a problem-solving mindset and a commitment to           │
│                            │ delivering high-quality, scalable, and efficient software solutions that meet      │
│                            │ business objectives. Eager to contribute technical expertise as well as innovative │
│                            │ thinking to a dynamic team.                                                        │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ mixtral-8x7B-instruct-v0.1 │ Executive Summary: Highly skilled and motivated Software Engineer with over 5      │
│                            │ years of experience in designing, developing, and maintaining complex software     │
│                            │ systems. Proven ability to work in a fast-paced, dynamic environment and           │
│                            │ collaborate effectively with cross-functional teams. Strong problem-solving skills │
│                            │ and expertise in a variety of programming languages, including Java, Python, and   │
│                            │ C++. Demonstrated ability to deliver high-quality software solutions on time and   │
│                            │ within budget. Committed to staying up-to-date with the latest industry trends and │
│                            │ best practices. Seeking an opportunity to leverage my skills and experience to     │
│                            │ contribute to the success of a forward-thinking organization.                      │
└────────────────────────────┴────────────────────────────────────────────────────────────────────────────────────┘

In [8]:
resumes = r_example.to_pandas()

In [9]:
resumes.columns

Index(['agent.agent_name', 'answer.example', 'iteration.iteration',
       'model.frequency_penalty', 'model.logprobs', 'model.max_new_tokens',
       'model.max_tokens', 'model.model', 'model.presence_penalty',
       'model.stopSequences', 'model.temperature', 'model.top_k',
       'model.top_logprobs', 'model.top_p', 'model.use_cache',
       'prompt.example_system_prompt', 'prompt.example_user_prompt',
       'raw_model_response.example_raw_model_response'],
      dtype='object')

In [10]:
resumes[['model.model','answer.example']]

,model.model,answer.example
0,gpt-3.5-turbo,As a seasoned software engineer with 5 years o...
1,gpt-4-1106-preview,Executive Summary: A dedicated and results-dri...
2,mixtral-8x7B-instruct-v0.1,Executive Summary: Highly skilled and motivate...


In [11]:
import pandas as pd

In [12]:
resumes_dict = pd.Series(resumes['answer.example'].values, index=resumes['model.model']).to_dict()

In [13]:
resumes_dict

{'gpt-3.5-turbo': 'As a seasoned software engineer with 5 years of experience, I have a proven track record of designing and implementing complex software solutions. Skilled in multiple programming languages such as Java, Python, and JavaScript, I have successfully delivered projects on time and within budget. With a strong background in software development methodologies and a passion for innovation, I am eager to contribute my expertise to a dynamic team.',
 'gpt-4-1106-preview': 'Executive Summary: A dedicated and results-driven software engineer with a proven track record of designing, implementing, and maintaining complex software systems. Possesses a strong foundation in computer science principles, coupled with hands-on experience in full-stack development, database management, and cloud technologies. Adept at collaborating with cross-functional teams and excelling in agile environments. Brings a problem-solving mindset and a commitment to delivering high-quality, scalable, and 

Here we draft some personas that we will instruct the selected LLMs to reference in reviewing the improved versions of the resume:

In [14]:
q_hr = QuestionFreeText(
    question_name = 'hr',
    question_text = 'Draft a persona of someone who works in human resources.'
)

In [15]:
q_se = QuestionFreeText(
    question_name = 'se',
    question_text = 'Draft a persona of a senior engineer.'
)

In [16]:
r_hr = q_hr.run()

In [17]:
r_se = q_se.run()

In [18]:
r_hr.select('answer.hr').print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .hr                                                                                                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Name: Samantha Jones                                                                                            │
│ Title: Human Resources Manager                                                                                  │
│ Industry: Technology                                                                                            │
│ Years of Experience: 8                                                                                          │
│ Education: Bachelor's degree in Human Resource Management                                                       │
│ Skills: Employee relations, recruitment, performance management, employment law, benefits administration,       │
│ conflict resolution, training and development                                                                   │
│ Personality Traits: Empathetic, strong communicator, approachable, problem-solver, ethical, organized           │
│ Hobbies: Volunteering at local nonprofits, reading leadership books, practicing yoga                            │
│ Career Goals: To create a workplace culture that fosters growth, satisfaction, and productivity, and to         │
│ eventually become a Director of Human Resources                                                                 │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [19]:
r_se.select('answer.se').print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .se                                                                                                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Name: Alex Reed                                                                                                 │
│ Age: 42                                                                                                         │
│ Profession: Senior Engineer                                                                                     │
│ Industry: Software Development                                                                                  │
│ Experience: 20 years                                                                                            │
│ Skills: Expert in software architecture, proficient in multiple programming languages (Java, C++, Python),      │
│ experienced with Agile and DevOps methodologies, strong background in system design and scalability.            │
│ Education: Master's degree in Computer Science                                                                  │
│ Certifications: Certified Scrum Master, AWS Certified Solutions Architect                                       │
│ Interests: Mentoring junior engineers, staying abreast of the latest technology trends, contributing to open    │
│ source projects.                                                                                                │
│ Personality Traits: Analytical thinker, problem-solver, team player, excellent communicator, leadership         │
│ qualities.                                                                                                      │
│ Career Goals: To lead innovative projects that push the boundaries of technology, to mentor the next generation │
│ of engineers, and to contribute to the strategic direction of the company.                                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

We use the personas to construct agents:

In [20]:
personas = ['', r_hr[0]['answer']['hr'], r_se[0]['answer']['se']]

In [21]:
agents = [
    Agent(traits={'role':'', 'persona':''}),
    Agent(traits={'role':'Human resources', 'persona':r_hr[0]['answer']['hr']}),
    Agent(traits={'role':'Senior engineer', 'persona':r_se[0]['answer']['se']})
]

In [22]:
agents

[Agent(traits = {'role': '', 'persona': ''}),
 Agent(traits = {'role': 'Human resources', 'persona': "Name: Samantha Jones\nTitle: Human Resources Manager\nIndustry: Technology\nYears of Experience: 8\nEducation: Bachelor's degree in Human Resource Management\nSkills: Employee relations, recruitment, performance management, employment law, benefits administration, conflict resolution, training and development\nPersonality Traits: Empathetic, strong communicator, approachable, problem-solver, ethical, organized\nHobbies: Volunteering at local nonprofits, reading leadership books, practicing yoga\nCareer Goals: To create a workplace culture that fosters growth, satisfaction, and productivity, and to eventually become a Director of Human Resources"}),
 Agent(traits = {'role': 'Senior engineer', 'persona': "Name: Alex Reed\nAge: 42\nProfession: Senior Engineer\nIndustry: Software Development\nExperience: 20 years\nSkills: Expert in software architecture, proficient in multiple programming 

Next we define some methods for improving the resume and then critiquing the improvements:

In [23]:
def improve(resume, model):
    q_improve = QuestionFreeText(
        question_name = "improve",
        question_text = "Draft an improved version of the following resume: " + resume
    )	
    r_improve = q_improve.by(model).run()
    return r_improve[0]['answer']['improve']

In [24]:
def score(resume, agent, model):
    q_score = QuestionLinearScale(
        question_name = "score",
        question_text = "Rank the following resume on a scale from 0 (lowest) to 10 (highest): " + resume,
        question_options = [0,1,2,3,4,5,6,7,8,9,10]
    )
    r_score = q_score.by(agent).by(model).run()
    return r_score[0]['answer']['score']

In [25]:
results = []

for drafting_model, resume in resumes_dict.items():
    
    for improving_model in models:
        improved_resume = improve(resume, improving_model)
    
        for scoring_model in models:
            for agent in agents:
                score_result = score(improved_resume, agent, scoring_model)
                            
                result = {
                    'drafting_model': drafting_model,
                    'improving_model': improving_model.model,
                    'scoring_model': scoring_model.model,
                    'score': score_result,
                    'persona': agent.traits['role']
                }
                results.append(result)

In [26]:
df = pd.DataFrame(results)

In [27]:
pd.set_option('display.max_rows', None) 
pd.set_option('display.width', 1000)
print(df)

                drafting_model             improving_model               scoring_model score          persona
0                gpt-3.5-turbo               gpt-3.5-turbo               gpt-3.5-turbo     9                 
1                gpt-3.5-turbo               gpt-3.5-turbo               gpt-3.5-turbo     8  Human resources
2                gpt-3.5-turbo               gpt-3.5-turbo               gpt-3.5-turbo     7  Senior engineer
3                gpt-3.5-turbo               gpt-3.5-turbo          gpt-4-1106-preview     8                 
4                gpt-3.5-turbo               gpt-3.5-turbo          gpt-4-1106-preview     8  Human resources
5                gpt-3.5-turbo               gpt-3.5-turbo          gpt-4-1106-preview     7  Senior engineer
6                gpt-3.5-turbo               gpt-3.5-turbo  mixtral-8x7B-instruct-v0.1     9                 
7                gpt-3.5-turbo               gpt-3.5-turbo  mixtral-8x7B-instruct-v0.1     8  Human resources
8         

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>